# 02-04-01 : Checkpoint

In [1]:
import pandas as pd
import numpy as np

import logging
from typing import Iterable
from tqdm.auto import tqdm

import jo_wilder

import keras


/root/miniconda3/envs/jo_wilder/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-18 12:18:42.021090: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Preparation & Cleaning

In [2]:
def map_question_to_level_group(question_number):
    """
    Maps the question number to the level group.

    Parameters
    ----------
    question_number : int
        The question number.

    Returns
    -------
    str
        The level group.
    """
    if question_number in [1, 2, 3]:
        return '0-4'
    elif question_number in [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return '5-12'
    elif question_number in [14, 15, 16, 17, 18]:
        return '13-22'
    else:
        return None

In [3]:
def prepare_label_dataset(data : pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the label dataset and add columns for the level group 
    and the question number.

    Parameters
    ----------
    data : pd.DataFrame
        The label dataset.

    Returns
    -------
    pd.DataFrame
        The prepared label dataset.
    """
    # add the columns to determine the level group
    df_labels = data \
        .rename(columns={'session_id': 'id'}) \
        .assign(session_id=lambda df: df['id'].str.split('_').str[0].astype(int)) \
        .assign(question_id=lambda df: df['id'].str.split('_').str[1]) \
        .assign(question_num=lambda df: df['question_id'].str[1:].astype(int)) \
        [['session_id', 'question_num', 'correct']]
    
    # add the level group column
    df_labels['level_group'] = df_labels['question_num'].apply(map_question_to_level_group) 
        
    return df_labels

In [4]:
def prepare_main_dataset(data : pd.DataFrame,
                         elapsed_time_min_clip:int=0,
                         elapsed_time_max_clip:int=3691298) -> pd.DataFrame:
    """
    Prepares the main dataset by removing duplicates and removing 
    columns that are not needed.

    Parameters
    ----------
    data : pd.DataFrame
        The main dataset.

    Returns
    -------
    pd.DataFrame
        The prepared main dataset.
    """
    empty_columns = ['fullscreen', 'hq', 'music', 'page', 'hover_duration']

    df_main = data \
        .drop_duplicates() \
        .reset_index(drop=True) \
        .drop(empty_columns, axis=1) \
        .drop('text', axis=1)
    
    # clip the elapsed time to remove outliers
    df_main['elapsed_time'] = df_main['elapsed_time'].clip(
        lower=elapsed_time_min_clip,
        upper=elapsed_time_max_clip)
    
    return df_main

## Feature Engineering

In [5]:
def create_initial_features(X:pd.DataFrame,
                            y:pd.DataFrame) -> pd.DataFrame:
    """
    Creates the initial dataset to which additional features will be added.

    Parameters
    ----------
    X : pd.DataFrame
        The main dataset.
    y : pd.DataFrame
        The label dataset.

    Returns
    -------
    pd.DataFrame
        The initial feature dataset.
    """
    df_features =  y \
        .groupby(['session_id', 'level_group']) \
        .agg({'correct': ['count']}) \
        .reset_index() \
        .droplevel(1, axis=1) \
        .drop(columns=['correct']) \
        .sort_values(['session_id', 'level_group']) \
        
    # set the session_id to be an integer
    df_features['session_id'] = df_features['session_id'].astype(int)
        
    return df_features        

In [6]:
def add_numeric_column_features(features:pd.DataFrame,
                                X:pd.DataFrame,
                                column:str,
                                min_values:dict=None,
                                max_values:dict=None) -> pd.DataFrame:
    """
    Add the maximum elapsed time feature to the features dataset.

    Parameters
    ----------
    features : pd.DataFrame
        The features dataset.
    X : pd.DataFrame
        The main dataset.
    column : str
        The name of the numeric column to add to the features for.

    Returns
    -------
    None
    """
    # Define a function to calculate mode
    def mode(series):
        return series.mode().iat[0]

    # calculate the maximum, minimum and mean for the column
    df_result = X \
        .groupby(['session_id', 'level_group']) \
        .agg({column: ['sum', 'max', 'min', 'mean', mode]}) \
        .reset_index()
    
    # flatten the multi-index columns
    df_result.columns = ['_'.join(col).rstrip('_') for col in df_result.columns.values]

    # normalize the values
    if min_values is None or max_values is None:
        logging.warning('Not normalizing the values, min_value and max_values are not set.')
    else:
        metric_list = ['sum', 'max', 'min', 'mean', 'mode']
        for metric in metric_list:
            current_column = f'{column}_{metric}'
            df_result[current_column] = (df_result[current_column] - min_values[metric]) / (max_values[metric] - min_values[metric])       

    # join the features to the result   
    df_result = features.set_index(['session_id', 'level_group']) \
        .join(df_result.set_index(['session_id', 'level_group']), how='left') \
        .reset_index()
    
    return df_result

In [7]:
def feature_engineering(df_source:pd.DataFrame,
                        df_source_labels:pd.DataFrame) -> pd.DataFrame:
    """
    Creates the feature dataset.

    Parameters
    ----------
    df_source : pd.DataFrame
        The main dataset.
    df_source_labels : pd.DataFrame
        The label dataset.

    Returns
    -------
    pd.DataFrame
        The feature dataset.
    """
    # create the initial features
    df_features = create_initial_features(df_source, df_source_labels)

    # add the feature to the features dataset
    df_features = add_numeric_column_features(
        features=df_features,
        X=df_source,
        column='elapsed_time',
        min_values={
            'sum': 61395.0,
            'max':  990.0,
            'min':  0.0,
            'mean': 526.447,
            'mode': 0.0},
        max_values={
            'sum':  9990648000,
            'max':  3691298.0,
            'min':  3691298.0,
            'mean': 3691298.0,
            'mode': 3691298.0})
    
    return df_features

## Data Selection

In [8]:
def create_feature_dataset(df_features:pd.DataFrame,
                           df_source_labels:pd.DataFrame,
                           session_list: list,
                           feature_list:list,
                           level_group:str=None,
                           include_question:bool=True,
                           expand_question:bool=False) -> np.array:
    """
    Creates the feature dataset for the given level group and session list.
    If the level group is not specified it will create the dataset for all level groups.

    Parameters
    ----------
    df_features : pd.DataFrame
        The dataset of prepared features (by session_id and level_group).
    df_source_labels : pd.DataFrame
        The dataset containing the training labels (y_True).
    session_list : list
        The list of session ids to create the dataset for.
    level_group : str, optional
        The level group to create the dataset for, by default None
    feature_list : list
        The list of features to include in the dataset.
    include_question : bool, optional
        Whether to include the question number in the dataset as the first set of
        columns, by default True
    expand_question : bool, optional
        Whether to expand the question number into a one-hot vector to each item in the 
        case of a multi-dimensional feature, by default False

    Returns
    -------
    np.array
        The feature dataset.
    """
    # get the features and labels for the given level group
    if level_group is None:
        logging.info('Creating the dataset for all level groups')
        df_features_group = df_features.query('session_id in @session_list')
        df_labels_group = df_source_labels.query('session_id in @session_list')
    else:
        logging.info('Creating the dataset for level group: %s', level_group)
        df_features_group = df_features.query('level_group == @level_group and session_id in @session_list')
        df_labels_group = df_source_labels.query('level_group == @level_group and session_id in @session_list')

    # sort the df_labels_group
    df_labels_group = df_labels_group.sort_values(['session_id', 'question_num'])

    feature_dataset = []

    # get the features for each row in the level group labels dataset
    current_session_id = None
    df_session_features = None

    for index, row in tqdm(df_labels_group.iterrows(), total=df_labels_group.shape[0]):        
        session_id = int(row['session_id'])
        session_level_group = row['level_group']
        question_num = int(row['question_num'])

        # get the features for the session
        if session_id != current_session_id:
            current_session_id = session_id
            df_session_features = df_features_group.query('session_id == @session_id')

        # get the level group features
        df_level_group_features = df_session_features.query('level_group == @session_level_group')

        # check if the session has features
        if df_level_group_features.shape[0] == 0:
            raise Exception(f'No features for session {session_id}, level group {session_level_group}!')
                            
        # get the features for the row
        row_features = []

        # get the question number one-hot encoded
        question_num_one_hot = np.zeros(18, dtype=np.int8)
        question_num_one_hot[question_num-1] = 1

        if include_question:
            row_features.extend(question_num_one_hot)

        for feature in feature_list:
            feature_value = df_level_group_features[feature].values[0]

            # check if the feature value is iterable
            if isinstance(feature_value, Iterable):
                if expand_question:
                    # reshape the question array to match the feature array shape
                    question_reshaped = np.tile(
                        question_num_one_hot, 
                        (feature_value.shape[0], 1))
                    
                    # add the question columns to the feature array
                    feature_value = np.hstack((question_reshaped, feature_value))

                row_features.extend(feature_value)
            else:
                row_features.append(feature_value)

        # add the row features to the output dataset
        feature_dataset.append(row_features)

    return np.array(feature_dataset, dtype=np.float32)

## Perform Predictions

In [9]:
# load the model
model_path = '../checkpoints/'
#model_path = '/kaggle/input/jw-02-04-91'

model = keras.models.load_model(f'{model_path}/02-04-01.h5')
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 23)]              0         
                                                                 
 dense_34 (Dense)            (None, 1024)              24576     
                                                                 
 dense_35 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 25,601
Trainable params: 25,601
Non-trainable params: 0
_________________________________________________________________


2023-03-18 12:18:43.646411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 12:18:43.647363: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
env = jo_wilder.make_env()
iter_test = env.iter_test()

In [11]:
def predict(model,
            X: pd.DataFrame,
            y: pd.DataFrame,
            threshold:float) -> pd.DataFrame:
    """
    Prepare the input dataset for prediction and return the predictions.

    Parameters
    ----------
    model : object
        The model to be used for prediction.
    X : pd.DataFrame
        The input dataset.
    y : pd.DataFrame
        The target dataset. Here the correct value should be updated

    Returns
    -------
    pd.DataFrame
        The updated y dataframe with the predictions.
    """
    feature_list = ['elapsed_time_sum', 'elapsed_time_max',
                    'elapsed_time_min', 'elapsed_time_mean',
                    'elapsed_time_mode']

    # prepare the main dataset
    df_source = prepare_main_dataset(X, elapsed_time_min_clip=0, elapsed_time_max_clip=3691298)

    # prepare the label dataset
    df_source_labels = prepare_label_dataset(y)

    # perform the feature engineering
    df_features = feature_engineering(df_source, df_source_labels)

    # create the dataset to be used for prediction
    X_pred = create_feature_dataset(
        df_features=df_features,
        df_source_labels=df_source_labels,
        session_list=df_source_labels['session_id'].unique(),
        feature_list=feature_list,
        level_group=None,
        include_question=True,
        expand_question=False)
    
    # get the predictions
    y_pred = (model.predict(X_pred) > threshold).astype(int)

    # update the y dataframe with the predictions
    df_pred = y \
        .copy() \
        .drop(columns=['correct']) \
        .assign(correct=y_pred)

    # display(df_source)
    # display(df_source_labels)
    # display(df_features)
    # display(X_pred)
    # display(y_pred)
    # display(df_pred)

    return df_pred

In [12]:
# delete the submission.csv file if
!rm submission.csv

In [13]:
for (df_label, df_data) in iter_test:
    df_predicted = predict(
        model=model,
        X=df_data, 
        y=df_label,
        threshold=0.47)
    env.predict(df_predicted)
    #break


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


100%|██████████| 3/3 [00:00<00:00, 766.22it/s]

1/1 [==============================] - 0s 53ms/step



100%|██████████| 3/3 [00:00<00:00, 685.31it/s]

1/1 [==============================] - 0s 18ms/step


...

1/1 [==============================] - 0s 16ms/step


100%|██████████| 5/5 [00:00<00:00, 642.53it/s]


1/1 [==============================] - 0s 20ms/step


100%|██████████| 5/5 [00:00<00:00, 770.47it/s]

1/1 [==============================] - 0s 18ms/step


In [14]:
# confirm submission file
df = pd.read_csv('submission.csv')
print(df.shape)
df.sample(n=10)

(54, 2)


,session_id,correct
6,20090312331414616_q1,1
11,20090109393214576_q6,0
23,20090312143683264_q8,0
31,20090312331414616_q6,1
53,20090312331414616_q18,1
46,20090312143683264_q16,1
52,20090312331414616_q17,1
10,20090109393214576_q5,0
39,20090109393214576_q14,1
33,20090312331414616_q8,1
